In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, date, timedelta, timezone
import pandas as pd
import numpy as np
import random
import warnings
warnings.simplefilter('ignore')
from scipy.stats import linregress

from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Iterable, Tuple

import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

try:
    from hmmlearn.hmm import GaussianHMM
except ImportError:
    GaussianHMM = None  # type: ignore

import matplotlib.pyplot as plt

def log_ret(s):                # daily log-return
    return np.log(s/ s.shift(1))

### IMB states

In [2]:
lengths = []
Y = np.array([[0,0]])

for t in ["BAC", "PEP", "KO", "UL", "WFC", "C", "GS", "MSFT", "AAPL", "BTC-USD", "ETH-USD", "TRX-USD", "JPM", "AAL", "WMT", "AMZN", "TSLA", "IBM", "INTC", "AMD", "SONY", "MCD", "^GSPC", "BABA", "PFE", "AAL", "F", "GM", "OXY", "CRM", "KR", "DIS", "EBAY", "USB", "GT", "DAL", "PM", "DDOG", "ADA-USD", "DOT-USD", "V", "CNP", "XYZ", "COIN", "VTRS", "COMM", "UAL", "BAX", "BA", "XRP-USD", "SOL-USD", "DOGE-USD", "AVAX-USD"]:

    data = yf.Ticker(t) #PEP, KO er utroligt stabile. NOVAQ, BAC er utroligt ustabil
    data = data.history(start = "1996-01-01", end = "2025-08-15")
    data = data.reset_index(drop = True)
    
    data = data[["Open", "High", "Low", "Close", "Volume"]]

    # returns
    data['returns'] = data['Close'].diff()/data['Close'].shift(1)
    data['returns'].loc[data['returns'].isna()] = 0
    
    # pn-counter
    pos_neg = []
    for j in range(len(data)):
        if data['returns'][j] >= 0:
            pos_neg.append(1)
        elif data['returns'][j] < 0:
            pos_neg.append(-1)
        else:
            pos_neg.append(0)
    data['PN_counter'] = pos_neg
    
    
    imb = [0]
    for i in range(1,len(data)):
        if i < 2000:
            imb.append(sum(data['PN_counter'][0:i] * data['Volume'][0:i]) / sum(data['Volume'][0:i]))
        else:
            imb.append(sum(data['PN_counter'][i-2000:i] * data['Volume'][i-2000:i]) / sum(data['Volume'][i-2000:i]))

    data["IMB"] = imb

    data = data.fillna(0)
    data["volume-state"] = data["Volume"].rolling(window=1000).mean()/np.mean(data["Volume"]) - 1
    data = data.fillna(0)
    
    #features needed 
    features = data[["IMB","volume-state"]][1000:]
    features = features.reset_index(drop = True)
    
    X = np.array(features)
    Y = np.concatenate([Y, X])
        
    lengths.append(len(features))

Y = Y[1:]
imb_model = GaussianHMM(n_components=3,covariance_type="full",n_iter=1000,random_state=2)
imb_model.fit(Y, lengths = lengths)

import pickle
from hmmlearn import hmm

# Assume `model` is your fitted HMM
# Save the model
with open("IMB.pkl", "wb") as f:
    pickle.dump(imb_model, f)

### Volatility states

In [3]:
# needs to be more precision

lengths = []
Y = np.array([[0,0,0,0]])

for t in ["BAC", "PEP", "KO", "UL", "WFC", "C", "GS", "MSFT", "AAPL", "BTC-USD", "ETH-USD", "TRX-USD", "JPM", "AAL", "WMT", "AMZN", "TSLA", "IBM", "INTC", "AMD", "SONY", "MCD", "^GSPC", "BABA", "PFE", "AAL", "F", "GM", "OXY", "CRM", "KR", "DIS", "EBAY", "USB", "GT", "DAL", "PM", "DDOG", "ADA-USD", "DOT-USD", "V", "CNP", "XYZ", "COIN", "VTRS", "COMM", "UAL", "BAX", "BA", "XRP-USD", "SOL-USD", "DOGE-USD", "AVAX-USD"]:

    data = yf.Ticker(t) #PEP, KO er utroligt stabile. NOVAQ, BAC er utroligt ustabil
    data = data.history(start = "1996-01-01", end = "2025-08-15")
    data = data.reset_index(drop = True)
    
    data = data[["Open", "High", "Low", "Close", "Volume"]]

    # returns
    data['returns'] = data['Close'].diff()/data['Close'].shift(1)
    data['returns'].loc[data['returns'].isna()] = 0

    data['volatility'] = data['returns'].rolling(window=20).std()
    data['VolAdjMomentum'] = data['returns'] / data['volatility']

    # ITR - interday true range
    data['ITR'] = (data['High'] - data['Low']) / (data['Open'])
    data['ITR'].loc[data['ITR'].isna()] = 0
    data['ITR'][np.isinf(data['ITR'])] = 0


    #features needed 
    features = data[["returns","volatility", "VolAdjMomentum", "ITR"]][1000:]
    features = features.reset_index(drop = True)
    
    X = np.array(features)
    Y = np.concatenate([Y, X])
        
    lengths.append(len(features))

Y = Y[1:]
vol_model = GaussianHMM(n_components=3,covariance_type="full",n_iter=1000,random_state=2)
vol_model.fit(Y, lengths = lengths)

import pickle
from hmmlearn import hmm

# Assume `model` is your fitted HMM
# Save the model
with open("VOL.pkl", "wb") as f:
    pickle.dump(vol_model, f)

### Returns

In [6]:
# needs to be more precision

lengths = []
Y = np.array([[0,0,0,0,0,0,0]])

for t in ["BAC", "PEP", "KO", "UL", "WFC", "C", "GS", "MSFT", "AAPL", "BTC-USD", "ETH-USD", "TRX-USD", "JPM", "AAL", "WMT", "AMZN", "TSLA", "IBM", "INTC", "AMD", "SONY", "MCD", "^GSPC", "BABA", "PFE", "AAL", "F", "GM", "OXY", "CRM", "KR", "DIS", "EBAY", "USB", "GT", "DAL", "PM", "DDOG", "ADA-USD", "DOT-USD", "V", "CNP", "XYZ", "COIN", "VTRS", "COMM", "UAL", "BAX", "BA", "XRP-USD", "SOL-USD", "DOGE-USD", "AVAX-USD"]:

    data = yf.Ticker(t) #PEP, KO er utroligt stabile. NOVAQ, BAC er utroligt ustabil
    data = data.history(start = "1996-01-01", end = "2025-08-15")
    data = data.reset_index(drop = True)
    
    data = data[["Open", "High", "Low", "Close", "Volume"]]

    # returns
    data['returns'] = data['Close'].diff()/data['Close'].shift(1)
    data['returns'].loc[data['returns'].isna()] = 0

    # log transform of returns
    data['log returns'] = log_ret(data['Close'])

    # Long-term returns
    data['returns1'] = data['Close'].diff()/data['Close'].shift(21)
    data['returns1'].loc[data['returns1'].isna()] = 0

    data['returns3'] = data['Close'].diff()/data['Close'].shift(63)
    data['returns3'].loc[data['returns3'].isna()] = 0

    data['returns6'] = data['Close'].diff()/data['Close'].shift(126)
    data['returns6'].loc[data['returns6'].isna()] = 0

    data['ma21'] = data['Close'].rolling(window=21).mean()
    data['ma63'] = data['Close'].rolling(window=63).mean()
    data['ma126']= data['Close'].rolling(window=126).mean()

    data["sma-s"] = np.sign(data["Close"] - data["ma21"])
    data["mma-s"] = np.sign(data["Close"] - data["ma63"])
    data["lma-s"] = np.sign(data["Close"] - data["ma126"])

    #features needed 
    features = data[["log returns", "returns1","returns3","returns6", "sma-s","mma-s","lma-s"]][1000:]
    features = features.reset_index(drop = True)
    
    X = np.array(features)
    Y = np.concatenate([Y, X])
        
    lengths.append(len(features))

Y = Y[1:]
ret_model = GaussianHMM(n_components=2,covariance_type="full",n_iter=1000,random_state=2)
ret_model.fit(Y, lengths = lengths)

import pickle

# Assume `model` is your fitted HMM
# Save the model
with open("ret.pkl", "wb") as f:
    pickle.dump(imb_model, f)

### Prices

In [4]:
# needs to be more precision

lengths = []
Y = np.array([[0,0,0,0,0,0]])

for t in ["BAC", "PEP", "KO", "UL", "WFC", "C", "GS", "MSFT", "AAPL", "BTC-USD", "ETH-USD", "TRX-USD", "JPM", "AAL", "WMT", "AMZN", "TSLA", "IBM", "INTC", "AMD", "SONY", "MCD", "^GSPC", "BABA", "PFE", "AAL", "F", "GM", "OXY", "CRM", "KR", "DIS", "EBAY", "USB", "GT", "DAL", "PM", "DDOG", "ADA-USD", "DOT-USD", "V", "CNP", "XYZ", "COIN", "VTRS", "COMM", "UAL", "BAX", "BA", "XRP-USD", "SOL-USD", "DOGE-USD", "AVAX-USD"]:

    data = yf.Ticker(t) #PEP, KO er utroligt stabile. NOVAQ, BAC er utroligt ustabil
    data = data.history(start = "1996-01-01", end = "2025-08-15")
    data = data.reset_index(drop = True)
    
    data = data[["Open", "High", "Low", "Close", "Volume"]]
    
    data['log returns'] = log_ret(data['Close'])

    data['ma20'] = data['Close'].rolling(window=20).mean()
    data['ma40'] = data['Close'].rolling(window=40).mean()
    data['ma120']= data['Close'].rolling(window=120).mean()

    # Compute MACD line and Signal line
    data['EMA20'] = data['Close'].ewm(span=20, adjust=False).mean()
    data['EMA40']  = data['Close'].ewm(span=40, adjust=False).mean()
    data['EMA120']  = data['Close'].ewm(span=120, adjust=False).mean()

    data["sma-s"] = np.sign(data["Close"] - data["ma20"])
    data["mma-s"] = np.sign(data["Close"] - data["ma40"])
    data["lma-s"] = np.sign(data["Close"] - data["ma120"])

    data["sEma-s"] = np.sign(data["Close"] - data['EMA20'])
    data["mEma-s"] = np.sign(data["Close"] - data["EMA40"])
    data["lEma-s"] = np.sign(data["Close"] - data["EMA120"])

    #features needed 
    features = data[["sma-s","mma-s","lma-s", "sEma-s", "mEma-s", "lEma-s"]][1000:]
    features = features.reset_index(drop = True)
    
    X = np.array(features)
    Y = np.concatenate([Y, X])
        
    lengths.append(len(features))

Y = Y[1:]
pp_model = GaussianHMM(n_components=3,covariance_type="full",n_iter=1000,random_state=3)
pp_model.fit(Y, lengths = lengths)

import pickle
from hmmlearn import hmm

# Assume `model` is your fitted HMM
# Save the model
with open("price.pkl", "wb") as f:
    pickle.dump(imb_model, f)

Model is not converging.  Current: 4507582.181299171 is not greater than 11298461.370097617. Delta is -6790879.188798445


# Test

In [7]:
class holdings:
    def __init__(self, symbols, inventory, allocations, riskiness):
        self.inventory = pd.DataFrame(columns = ["symbol","value","real inventory","inventory","max inventory","allocation","riskiness", "urgency"])
        
        if len(symbols) == len(inventory) and len(inventory) == len(allocations) and len(allocations) == len(riskiness):
            for i in range(len(symbols)):
                self._i = [symbols[i], allocations[i], 0,inventory[i],1,allocations[i],riskiness[i],0]
                self.inventory.loc[i] = self._i
                
                
    def post_order_size(self, symbol, lot):
        
        data = self.inventory[self.inventory["symbol"] == symbol]
        
        urg = data["urgency"].iloc[0]
        risk = data["riskiness"].iloc[0]
        inv = data["inventory"].iloc[0]
        max_inv = data["max inventory"].iloc[0]
        if max_inv != (1-risk) * data["allocation"].iloc[0]:
            max_inv = (1-risk) * data["allocation"].iloc[0]
            self.inventory.iloc[data.index[0],4] = (1-risk) * data["allocation"].iloc[0]
        
        ideal_inv = max_inv
        ideal_buy = max_inv + 1
        ideal_sell = max_inv - 1
        
        buy_size = max(max((ideal_buy - inv),0) * min(((urg+0.5)*lot),1) - 1,0)
        sell_size = min(min((ideal_sell - inv), 0) * min(((urg+0.5)*lot),1) + 1,0)
        
        return(buy_size, sell_size)
    
    def update_parameters(self, symbol, risk = None, urgency = None):
        
        data = self.inventory[self.inventory["symbol"] == symbol]
        
        if risk != None:
            self.inventory.iloc[data.index[0],6] = risk
            self.inventory.iloc[data.index[0],4] = (1-risk) * data["allocation"].iloc[0]
        if urgency != None:
            self.inventory.iloc[data.index[0],7] = urgency
            

### data

In [258]:
data = yf.Ticker("C") #PEP, KO er utroligt stabile. NOVAQ, BAC er utroligt ustabil
data = data.history(start = "1990-01-01", end = "2025-10-01")
data = data.reset_index(drop = True)

data = data[["Open", "High", "Low", "Close", "Volume"]]

# returns
data['returns'] = data['Close'].diff()/data['Close'].shift(1)
data['returns'].loc[data['returns'].isna()] = 0
# log transform of returns
data['log returns'] = log_ret(data['Close'])

# pn-counter
pos_neg = []
for j in range(len(data)):
    if data['returns'][j] >= 0:
        pos_neg.append(1)
    elif data['returns'][j] < 0:
        pos_neg.append(-1)
    else:
        pos_neg.append(0)
data['PN_counter'] = pos_neg

imb = [0]
for i in range(1,len(data)):
    if i < 2000:
        imb.append(sum(data['PN_counter'][0:i] * data['Volume'][0:i]) / sum(data['Volume'][0:i]))
    else:
        imb.append(sum(data['PN_counter'][i-2000:i] * data['Volume'][i-2000:i]) / sum(data['Volume'][i-2000:i]))

data["IMB"] = imb
data = data.fillna(0)
data["volume-state"] = data["Volume"].rolling(window=1000).mean()/np.mean(data["Volume"]) - 1

data['ma20'] = data['Close'].rolling(window=20).mean()
data['ma40'] = data['Close'].rolling(window=40).mean()
data['ma120']= data['Close'].rolling(window=120).mean()

# Compute MACD line and Signal line
data['EMA20'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA40']  = data['Close'].ewm(span=40, adjust=False).mean()
data['EMA120']  = data['Close'].ewm(span=120, adjust=False).mean()

data["sma-s"] = np.sign(data["Close"] - data["ma20"])
data["mma-s"] = np.sign(data["Close"] - data["ma40"])
data["lma-s"] = np.sign(data["Close"] - data["ma120"])

data["sEma-s"] = np.sign(data["Close"] - data['EMA20'])
data["mEma-s"] = np.sign(data["Close"] - data["EMA40"])
data["lEma-s"] = np.sign(data["Close"] - data["EMA120"])

# Long-term returns
data['returns1'] = data['Close'].diff()/data['Close'].shift(21)
data['returns1'].loc[data['returns1'].isna()] = 0

data['returns3'] = data['Close'].diff()/data['Close'].shift(63)
data['returns3'].loc[data['returns3'].isna()] = 0

data['returns6'] = data['Close'].diff()/data['Close'].shift(126)
data['returns6'].loc[data['returns6'].isna()] = 0

data['ma21'] = data['Close'].rolling(window=21).mean()
data['ma63'] = data['Close'].rolling(window=63).mean()
data['ma126']= data['Close'].rolling(window=126).mean()

data["rsma-s"] = np.sign(data["Close"] - data["ma21"])
data["rmma-s"] = np.sign(data["Close"] - data["ma63"])
data["rlma-s"] = np.sign(data["Close"] - data["ma126"])

data['volatility'] = data['returns'].rolling(window=20).std()
data['VolAdjMomentum'] = data['returns'] / data['volatility']

# ITR - interday true range
data['ITR'] = (data['High'] - data['Low']) / (data['Open'])
data['ITR'].loc[data['ITR'].isna()] = 0
data['ITR'][np.isinf(data['ITR'])] = 0

#imbalance states
imb_features = data[["IMB","volume-state"]][1000:] # check
imb_features = imb_features.reset_index(drop = True)

#return states
r_features = data[["log returns", "returns1","returns3","returns6", "rsma-s","rmma-s","rlma-s"]][1000:]
r_features = r_features.reset_index(drop = True)

#volatility states
v_features = data[["returns","volatility", "VolAdjMomentum", "ITR"]][1000:]
v_features = v_features.reset_index(drop = True)

#price states
p_features = data[["sma-s","mma-s","lma-s", "sEma-s", "mEma-s", "lEma-s"]][1000:] # check
p_features = p_features.reset_index(drop = True)

#adding states
data = data[1000:]
data["IMB_state"] = imb_model.predict(imb_features)
data["ret_state"] = ret_model.predict(r_features)
data["vol_state"] = vol_model.predict(v_features)
data["price_state"] = pp_model.predict(p_features)
data = data.reset_index(drop = True)

## buy

In [259]:
# down trend
b = data[data["price_state"] == 0]
#b = b[b["IMB"] > 0]
b["dist"] = abs(b["Close"] - b["ma21"])
b_s = b[b["dist"] > (b["dist"].rolling(100).mean() + 0.25 * b["dist"].rolling(100).std())]
#b_s = b_s[b_s["returns"].rolling(10).sum() < 0.1]
#b_s = b_s[b_s["ITR"] < b_s["ITR"].rolling(3).mean()]

#d = data[1000:]
#d = d.reset_index(drop = True)
qt = data["Close"].rolling(100).quantile(0.4)
lst = []
for i in range(len(qt)):
    if qt.index[i] in b_s.index:
        lst.append(qt[i])
        
b_s["QT"] = lst
b_s = b_s[b_s["Close"] < b_s["QT"]]
b_s

,Open,High,Low,Close,Volume,returns,log returns,PN_counter,IMB,volume-state,...,rlma-s,volatility,VolAdjMomentum,ITR,IMB_state,ret_state,vol_state,price_state,dist,QT
1136,158.509597,158.509597,154.474220,154.635635,936980,-0.038153,-0.038900,-1,0.100614,-0.963407,...,1.0,0.018300,-2.084855,0.025458,0,0,1,0,7.832467,155.025854
1137,154.635605,156.088340,152.698625,154.958435,953540,0.002087,0.002085,1,0.099852,-0.963362,...,1.0,0.018318,0.113956,0.021921,0,0,1,0,7.186839,155.153534
1171,170.776522,171.423402,159.779549,159.941269,1578940,-0.053588,-0.055078,-1,0.112552,-0.962117,...,-1.0,0.028694,-1.867568,0.068182,0,0,0,0,16.262044,160.059195
1387,169.591531,170.571829,161.259000,165.180191,839107,-0.021771,-0.022011,-1,0.099438,-0.935426,...,1.0,0.025405,-0.856959,0.054913,0,0,1,0,6.916582,165.194336
1420,180.415851,181.890640,175.008291,175.254089,1242800,-0.025956,-0.026299,-1,0.095381,-0.933590,...,1.0,0.014408,-1.801456,0.038147,0,0,1,0,12.639499,179.614291
1421,176.728838,180.415809,173.041866,173.533463,1262547,-0.009818,-0.009866,-1,0.094572,-0.933534,...,-1.0,0.014127,-0.694991,0.041725,0,0,1,0,13.567539,179.614291
1422,176.728838,176.728838,173.287664,173.533463,912427,0.000000,0.000000,1,0.093628,-0.933485,...,-1.0,0.014147,0.000000,0.019471,0,0,1,0,12.693255,179.614291
1559,214.036431,217.002293,209.834792,212.059189,910360,-0.022779,-0.023043,-1,0.101190,-0.923997,...,1.0,0.024125,-0.944219,0.033487,0,0,1,0,9.424030,212.704086
1560,214.530744,216.507986,206.374621,207.610397,1287480,-0.020979,-0.021202,-1,0.100303,-0.923979,...,1.0,0.024187,-0.867380,0.047235,0,0,1,0,12.538749,212.704086
1561,207.363221,208.351841,197.477012,203.408737,2039120,-0.020238,-0.020446,-1,0.099477,-0.923894,...,1.0,0.023292,-0.868894,0.052443,0,0,1,0,15.420462,212.704086


## sell

In [260]:
c = data[data["price_state"] == 1]
c = c[c["IMB"] > 0]
c["dist"] = abs(c["Close"] - c["ma21"])
c_s = c[c["dist"] > (c["dist"].rolling(200).mean() + 0.5*c["dist"].rolling(200).std())]
c_s = c_s[c_s["returns"].rolling(10).sum() > 0]

c_s = c_s[c_s["ITR"] < c_s["ITR"].rolling(3).mean()]

qt = data["Close"].rolling(100).quantile(0.5)
lst = []
for i in range(len(qt)):
    if qt.index[i] in c_s.index:
        lst.append(qt[i])

c_s["QT"] = lst
c_s = c_s[c_s["Close"] > c_s["QT"]]
c_s

,Open,High,Low,Close,Volume,returns,log returns,PN_counter,IMB,volume-state,...,rlma-s,volatility,VolAdjMomentum,ITR,IMB_state,ret_state,vol_state,price_state,dist,QT
483,47.308850,47.518184,46.366859,46.366859,643860,-0.030635,-0.031114,-1,0.126705,-0.968719,...,1.0,0.021325,-1.436572,0.024336,1,0,1,1,2.627291,40.447638
487,45.006203,45.634196,44.587539,45.529530,513060,0.023529,0.023257,1,0.124321,-0.968684,...,1.0,0.022676,1.037629,0.023256,1,0,1,1,1.762033,41.205210
488,45.948209,46.262206,45.843542,46.052872,468120,0.011495,0.011429,1,0.125062,-0.968673,...,1.0,0.021101,0.544728,0.009112,1,0,1,1,2.167693,41.287792
497,49.402167,50.030160,49.402167,49.716164,825900,-0.002101,-0.002103,-1,0.128884,-0.968509,...,1.0,0.026353,-0.079727,0.012712,1,0,1,1,3.762980,41.881239
500,52.332791,52.332791,51.704797,51.914131,679200,-0.006013,-0.006031,-1,0.129761,-0.968448,...,1.0,0.026045,-0.230851,0.012000,1,0,1,1,5.008990,42.141474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7990,98.779999,99.699997,98.440002,99.440002,9502900,0.006274,0.006254,1,0.001881,0.327605,...,1.0,0.012857,0.488000,0.012756,2,0,2,1,3.723812,85.467899
7991,99.800003,100.169998,99.400002,99.790001,10620100,0.003520,0.003514,1,0.001646,0.325643,...,1.0,0.011901,0.295740,0.007715,2,0,2,1,3.867620,85.934830
7994,101.970001,102.699997,101.690002,102.410004,11677600,0.006388,0.006367,1,0.001573,0.323753,...,1.0,0.011653,0.548143,0.009905,2,0,2,1,5.384765,86.719666
7995,102.540001,103.129997,101.870003,102.680000,24745700,0.002636,0.002633,1,0.001567,0.324420,...,1.0,0.011377,0.231734,0.012288,2,0,2,1,5.201905,86.977962


## run!

In [261]:
hold = holdings(["BAC"], [0], [100], [0])

lot = 1

for i in range(len(data)):
    
    #lot = min(max(0, lot),1)
    
    # order size
    os = list(hold.post_order_size("BAC", lot))
    
    # inventory adjustment
    # hold.inventory.iloc[0,3] = hold.inventory.iloc[0,3] + sum(os) 
    
    # percentage change with order size
    pct = sum(os) / hold.inventory.iloc[0,5] 
    
    #portfolio value
    val_sum = hold.inventory.iloc[0,1] + hold.inventory.iloc[0,2] * data["Close"][i]
    
    # percentage of entire value
    pct_change = pct * val_sum
    if hold.inventory.iloc[0,1] - pct_change < 0:
        pct_change = hold.inventory.iloc[0,1]
    
    if i in c_s.index: # sell
        #pct = pct - 0.000005 #* min(risk+urgency,1)
        # inventory adjustment
        hold.inventory.iloc[0,3] = hold.inventory.iloc[0,3] + sum(os) 
        
        hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] + abs(pct_change) 
        hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][i]
    
    elif i in b_s.index: # buy
        #pct = pct + 0.000005 #* min(risk+urgency,1)# inventory adjustment
        hold.inventory.iloc[0,3] = hold.inventory.iloc[0,3] + sum(os) 
    
        hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] - pct_change
        hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][i]
    
    
    #if pct_change < 0: # sell
        
        #hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] + abs(pct_change) 
        #hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][i]
    
    #elif pct_change > 0: # buy
        
        #hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] - pct_change
        #hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][i]

    
    #elif os[1] < 0 and hold.inventory.iloc[0,6] > 0.3 and hold.inventory.iloc[0,7] > 0.3: # emergency sell

        
    #elif os[1] < 0 and data["price_state"][i] == 2: #sell


        
        
display(hold.inventory)
print(val_sum)

,symbol,value,real inventory,inventory,max inventory,allocation,riskiness,urgency
0,BAC,287.005117,3.962183,99.0,100,100,0,0


689.1667201239509


In [262]:
(val_sum/100)**(252/len(data)) - 1 #> 0.1704456701866821

0.06266729885146138

In [263]:
hold = holdings(["BAC"], [0], [100], [0])
# order size
os = [100,0]

# inventory adjustment
hold.inventory.iloc[0,3] = hold.inventory.iloc[0,3] + sum(os) 

# percentage change with order size
pct = sum(os) / hold.inventory.iloc[0,5] 

#portfolio value
val_sum = hold.inventory.iloc[0,1] + hold.inventory.iloc[0,2] * data["Close"][1]

#
pct_change = pct * val_sum

if pct_change < 0:
    hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] + abs(pct_change)
    hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][1]

elif pct_change > 0:
    hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] - pct_change
    hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][1]
    
    
# order size
os = [0,-100]

# inventory adjustment
hold.inventory.iloc[0,3] = hold.inventory.iloc[0,3] + sum(os) 

# percentage change with order size
pct = sum(os) / hold.inventory.iloc[0,5] 

#portfolio value
val_sum = hold.inventory.iloc[0,1] + hold.inventory.iloc[0,2] * data["Close"][len(data)-1]

#
pct_change = pct * val_sum

if pct_change < 0:
    hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] + abs(pct_change)
    hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][len(data)-1]

elif pct_change > 0:
    hold.inventory.iloc[0,1] = hold.inventory.iloc[0,1] - pct_change
    hold.inventory.iloc[0,2] = hold.inventory.iloc[0,2] + pct_change / data["Close"][len(data)-1]
    
    
display(hold.inventory)
print(val_sum)

,symbol,value,real inventory,inventory,max inventory,allocation,riskiness,urgency
0,BAC,321.987659,0.0,0,1,100,0,0


321.98765869614294


In [264]:
(val_sum/100)**(252/len(data)) - 1

0.037506769390061256

In [ ]:
lot = 1
    risk = 0
    urgency = 0
    # low risk
    
    # standard returns
    if data["ret_state"][i] == 0:
        lot = lot + 0.1
        
        # demand is stable and high
        if data["IMB_state"][i] == 1:
            lot = lot + 0.2
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = 1
                risk = 0
                urgency = 0
        
        # demand is stable but falling?
        elif data["IMB_state"][i] == 2:
            risk = risk + 0.25
            urgency = urgency + 0.1
            lot = lot - 0.3
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = lot
                risk = risk
                urgency = risk
        
        # demand is low and falling?
        elif data["IMB_state"][i] == 0:
            risk = risk + 0.75
            urgency = urgency + 0.5
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = lot
                risk = risk
                urgency = risk
            
                
    
    # exceptional returns
    elif data["ret_state"][i] == 1:
        urgency = urgency + 0.5
        lot = lot + 0.5
        
        # demand is stable and high
        if data["IMB_state"][i] == 1:
            lot = lot - 0.2
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = lot
                risk = risk
                urgency = risk
        
        # demand is stable but falling?
        elif data["IMB_state"][i] == 2:
            risk = risk + 0.25
            urgency = urgency + 0.1
            lot = lot - 0.3
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = lot
                risk = risk
                urgency = risk
        
        # demand is low and falling?
        elif data["IMB_state"][i] == 0:
            risk = risk + 0.75
            urgency = urgency + 0.5
            
            if data["vol_state"][i] == 1:
                risk = risk + 0.3
                urgency = urgency + 0.3
                lot = lot - 0.5
                
            elif data["vol_state"][i] == 0:
                risk = risk + 0.2
                urgency = urgency + 0.3
                lot = lot - 0.25

            elif data["vol_state"][i] == 2:
                lot = lot
                risk = risk
                urgency = risk
        
    
    update_risk = min(max(0, risk),1)
    update_urgency = min(max(0, urgency),1)
    hold.update_parameters("BAC", risk = update_risk, urgency = update_urgency)